In [2]:
import pandas as pd
import numpy as np

In [12]:
df = pd.read_csv("iowa-sample.csv")
print(df.shape)
df.head()

(1000, 24)


,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,S08096000008,10/04/2012,4641,Kum & Go #573 / SE 14th DM,5830 SE 14th ST,DES MOINES,50315,NaN,77,Polk,...,89191,Jose Cuervo Especial Reposado Tequila Mini,12,500,11.50,17.25,1,17.25,0.50,0.13
1,S23102300041,12/20/2014,4346,Roy's Foodland,105 PEARL ST,SHELLSBURG,52332,POINT (-91.869285 42.094155),6,Benton,...,11774,Black Velvet,24,375,3.07,4.61,6,27.66,2.25,0.59
2,S14410500035,09/09/2013,3628,Wal-Mart 1528 / Cedar Rapids,2645 BLAIRS FERRY RD NE,CEDAR RAPIDS,52402,POINT (-91.680734 42.034748),57,Linn,...,43410,Captain Morgan Parrot Bay Coconut,12,750,7.49,11.23,12,134.76,9.00,2.38
3,S09427600021,12/12/2012,4708,No Frills Supermarkets #803 / Glenwo,423 SHARP ST,GLENWOOD,51534,POINT (-95.742987 41.04635),65,Mills,...,24458,Kessler Blend Whiskey,6,1750,11.01,16.52,30,495.60,52.50,13.87
4,S28446900114,10/14/2015,2594,Hy-Vee Food Store / Sioux City,4500 SERGEANT ROAD,SIOUX CITY,51106,POINT (-96.346969 42.447396),97,Woodbury,...,10550,Black Velvet Toasted Caramel,12,750,6.75,10.13,12,121.56,9.00,2.38
